## Boto3 를 활용한 Opensearch domain 생성

- Opensearch domain은 console 에서 생성하는 것도 가능하지만, AWS SDK로 CLI를 사용해서 하는 것도 가능하며 boto3 (AWS Python SDK) 를 활용하여 쉽게 생성하는 것도 가능합니다.
- 아래는 boto3를 활용하여 생성하는 예시입니다.
- 자세한 내용은 [공식 문서 튜토리얼](https://docs.aws.amazon.com/ko_kr/opensearch-service/latest/developerguide/osis-sdk.html) 이나 [API 문서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/opensearch/client/create_domain.html) 를 참고해 주세요.

In [ ]:
import boto3
import json
import sagemaker

In [ ]:
client = boto3.client('opensearch')

In [ ]:
domain_name = "rag-opensearch-03"

In [ ]:
sess = sagemaker.session.Session()
region = sess._region_name

In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

access_policy_default = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "AWS": "*"
      },
      "Action": "es:*",
      "Resource": f"arn:aws:es:{region}:{account_id}:domain/{domain_name}/*"
    }
  ]
}

access_policy = json.dumps(access_policy_default)

In [ ]:
print(access_policy)

In [ ]:
response = client.create_domain(
    DomainName=domain_name,
    EngineVersion='OpenSearch_2.7',
    ClusterConfig={
        'InstanceType': 'r6g.large.search',
        'InstanceCount': 1,
        'DedicatedMasterEnabled': False,  # production에서는 True로 하는것이 좋습니다.
    },
    EBSOptions={
        'EBSEnabled': True,
        'VolumeType': 'gp3',
        'VolumeSize': 100
    },
    AdvancedSecurityOptions={
        'Enabled': True,
        'InternalUserDatabaseEnabled': True,
        'MasterUserOptions': {
            'MasterUserName': 'raguser',
            'MasterUserPassword': 'QWEqwe123!@#'
        }
    },
    AccessPolicies=access_policy,
    EncryptionAtRestOptions={
        'Enabled': True,
    },
    NodeToNodeEncryptionOptions={
        'Enabled': True
    },
    DomainEndpointOptions={
        'EnforceHTTPS': True,
    },
)

In [ ]:
print(response)

In [ ]:
response = client.describe_domain(
    DomainName=domain_name
)

In [ ]:
opensearch_endpoint = response['DomainStatus']['Endpoint']
print(opensearch_endpoint)